In [26]:
using JuMP
using HiGHS
using CSV
using DataFrames

In [27]:
data = CSV.read("gerrymander.csv", DataFrame)

Row,county_number,county_name,current_district,vote_diff_d_minus_r,vote_diff_d_minus_r_scenario_2,vote_diff_d_minus_r_scenario_3
,Int64,String15,Int64,Int64,Int64,Int64
1,1,Bernalillo,1,42941,43411,11336
2,2,Catron,2,-917,18,-716
3,3,Chaves,2,-6650,-6244,-6436
4,4,Cibola,2,1941,1449,1025
5,5,Colfax,3,116,-871,-1099
6,6,Curry,3,-5194,-4241,-5093
7,7,DeBaca,2,-299,223,567
8,8,Dona Ana,2,9790,8856,8251
9,9,Eddy,2,-6436,-6787,-6736


In [28]:
counties = data.county_number
districts = 1:3
vote_diff = Dict(row.county_number => row.vote_diff_d_minus_r for row in eachrow(data))

Dict{Int64, Int64} with 33 entries:
  5  => 116
  16 => 395
  20 => -5504
  12 => -66
  24 => 2707
  28 => -965
  8  => 9790
  17 => -81
  30 => 9145
  1  => 42941
  19 => 1361
  22 => 8016
  23 => -2313
  6  => -5194
  32 => -760
  11 => 870
  9  => -6436
  31 => -1107
  14 => -8412
  3  => -6650
  29 => 1285
  7  => -299
  25 => -13091
  33 => 685
  4  => 1941
  ⋮  => ⋮

In [29]:
model = Model(HiGHS.Optimizer)
@variable(model, x[districts, counties], Bin)
@objective(model, Max, sum(vote_diff[j] * x[2, j] for j in counties))
for j in counties
    @constraint(model, sum(x[i, j] for i in districts) == 1)
end
@constraint(model, sum(vote_diff[j] * x[1, j] for j in counties) >= 100)  # District 1
@constraint(model, sum(vote_diff[j] * x[2, j] for j in counties) >= 100)  #District 2
@constraint(model, sum(vote_diff[j] * x[3, j] for j in counties) >= 100)  # District 3

42941 x[3,1] - 917 x[3,2] - 6650 x[3,3] + 1941 x[3,4] + 116 x[3,5] - 5194 x[3,6] - 299 x[3,7] + 9790 x[3,8] - 6436 x[3,9] + 1723 x[3,10] + 870 x[3,11] - 66 x[3,12] + 99 x[3,13] - 8412 x[3,14] - 3009 x[3,15] + 395 x[3,16] - 81 x[3,17] + 9943 x[3,18] + 1361 x[3,19] - 5504 x[3,20] - 812 x[3,21] + 8016 x[3,22] - 2313 x[3,23] + 2707 x[3,24] - 13091 x[3,25] + 6473 x[3,26] + 34523 x[3,27] - 965 x[3,28] + 1285 x[3,29] + 9145 x[3,30] - 1107 x[3,31] - 760 x[3,32] + 685 x[3,33] ≥ 100

In [30]:
optimize!(model)

# Check if the solution is optimal
if termination_status(model) == MOI.OPTIMAL
    println("Optimal solution found!")
    println("Democratic margin in District 2: ", objective_value(model))

    # Output the county assignments for each district
    for i in districts
        assigned_counties = [j for j in counties if value(x[i, j]) > 0.5]
        println("District $i: Counties $assigned_counties")
    end
else
    println("No optimal solution found.")
end

Running HiGHS 1.8.0 (git hash: fcfb53414): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e+00, 4e+04]
  Cost   [7e+01, 4e+04]
  Bound  [1e+00, 1e+00]
  RHS    [1e+00, 1e+02]
Presolving model
36 rows, 99 cols, 198 nonzeros  0s
36 rows, 99 cols, 198 nonzeros  0s
Objective function is integral with scale 1

Solving MIP model with:
   36 rows
   99 cols (99 binary, 0 integer, 0 implied int., 0 continuous)
   198 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   132013          -inf                 inf        0      0      0         0     0.0s
 R       0       0         0   0.00%   76197           68266             11.62%        0      0      0        54     0.0s
 C       0       0         0   0.00%   76197           